In [1]:
import seaborn as sns
import numpy as np
import nibabel as nib
from nilearn import datasets
from statsmodels.stats.multitest import multipletests
from src.custom_plotting import plot_surface_stats, mkNifti
from nilearn import surface
from pathlib import Path

In [2]:
def load_noise_ceiling(mask_dir, sid, mask, dataset):
    noise_ceiling = np.load(f'{mask_dir}/sub-{sid}_set-{dataset}_stat-rho_statmap.npy')
    noise_ceiling = noise_ceiling[mask]
    noise_ceiling[noise_ceiling <= 0.] = np.nan
    return noise_ceiling

In [3]:
def filter_r(rs, ps, p_crit=0.05, correct=False, threshold=False):
    if correct and (ps is not None):
        _, ps_corrected, _, _ = multipletests(ps, method='fdr_bh')
    elif not correct and (ps is not None):
        ps_corrected = ps.copy()
    else:
        ps_corrected = None

    if threshold and (ps is not None):
        rs[ps_corrected >= p_crit] = 0.
    else:
        rs[rs < 0.] = 0.
    return rs, ps_corrected

In [4]:
def plot(r_, p_, sid, mask_dir, mask, img, fsaverage, method):
    r, _ = filter_r(r_, p_)
    if method == 'test':
        dataset = 'test'
    else:
        dataset = 'train'
    # noise_ceiling = load_noise_ceiling(mask_dir, sid, mask, dataset)**2
    # r_normed = np.divide(r_, noise_ceiling)
    # r_normed[np.isnan(r_normed)] = 0.
    r_normed = r.copy()
    volume = mkNifti(r_normed, mask, img)
    texture = {'left': surface.vol_to_surf(volume, fsaverage['pial_left']),
               'right': surface.vol_to_surf(volume, fsaverage['pial_right'])}

    # plotting.plot_glass_brain(volume,
    #                           threshold=1e-2, vmax=1.,
    #                           colorbar=True, symmetric_cbar=False,
    #                           cmap=cmap,
    #                           output_file=f'{figure_dir}/sub-{sid}_volume.pdf')

    # vmax = 1.
    threshold = np.nanmin(r_normed[np.invert(np.isclose(r_normed, 0.))])
    vmax = np.nanmax(r_normed)
    print(threshold)
    plot_surface_stats(fsaverage, texture,
                       roi=roi_parcel,
                       cmap=cmap,
                       modes=['lateral'],
                       threshold=threshold,
                       vmax=vmax,
                       cbar_title=r"Noise-normalized explained variance ($r^2$)",
                       output_file=f'{figure_dir}/sub-{sid}.pdf')

In [5]:
data_dir='/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/raw'
out_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim'
mask_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/data/interim/Reliability'
top_fig_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_analysis/reports/figures/PlotVoxelEncoding'

fsaverage = datasets.fetch_surf_fsaverage()
cmap = sns.color_palette('magma', as_cmap=True)
models = ['facingness', 'joint action', 'communication']#, 'annotated', 'social', 'socialprimitive', 'visual']
roi_parcel = None
img = nib.load(f'{mask_dir}/sub-01_set-test_stat-rho_statmap.nii.gz')
n_perm = 5000
n_subjs = 4
method = 'CV'

In [6]:
for sid in ['01', '02', '03', '04']:
    for model in models:
        print(f'sub-{sid}: {model}')
        figure_dir = f'{top_fig_dir}/{method}/{model}'
        Path(figure_dir).mkdir(parents=True, exist_ok=True)

        r_cond = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-{model}_method-{method}_rs.npy')
        r_all = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-all_method-{method}_rs.npy')
        r_unique = r_all**2 - r_cond**2
        p = np.load(f'{out_dir}/VoxelPermutation/sub-{sid}_prediction-{model}_method-{method}_ps.npy')
        mask = np.load(f'{out_dir}/Reliability/sub-all_set-test_reliability-mask.npy').astype('bool')
        plot(r_unique, p, sid, mask_dir, mask, img, fsaverage, method)

sub-01: facingness
3.2508483365819204e-08
sub-01: joint action
8.614548439133276e-08
sub-01: communication
1.068011890572742e-07
sub-02: facingness
4.1594640938511434e-08
sub-02: joint action
2.7522255625142437e-07
sub-02: communication
1.4217637527876156e-08
sub-03: facingness
1.7491541286006562e-08
sub-03: joint action
8.29469431212369e-08
sub-03: communication
1.1201631982225846e-07
sub-04: facingness
1.854888133775749e-08
sub-04: joint action
1.9870784970436877e-08
sub-04: communication
2.3907065521255183e-08


## CV

sub-all: all
0.0631975360498012
sub-all: annotated
0.06852206426895464
sub-all: social
0.06733947858672726
sub-all: socialprimitive
0.061688830459653766
sub-all: visual
0.07508201780010568
sub-01: all
0.19710323592952353
sub-01: annotated
0.21299163933680912
sub-01: social
0.24851814420135068
sub-01: socialprimitive
0.3475932623463479
sub-01: visual
0.2581237118488072
sub-02: all
0.16106848173399038
sub-02: annotated
0.1575668065152995
sub-02: social
0.18176281495486762
sub-02: socialprimitive
0.2684050238495838
sub-02: visual
0.22039256562543508
sub-03: all
0.17644936721341456
sub-03: annotated
0.18059337314134882
sub-03: social
0.1993087680337161
sub-03: socialprimitive
0.19075753339992757
sub-03: visual
0.17793348987720783
sub-04: all
0.14438412074624118
sub-04: annotated
0.11520621087218676
sub-04: social
0.2444177936439139
sub-04: socialprimitive
0.11475725752040819
sub-04: visual
0.19607340970603424

#Test

sub-01: all
0.43543420107836805
sub-01: annotated
0.4570510498675174
sub-01: social
0.5366723383283916
sub-01: socialprimitive
0.5525512352336553
sub-01: visual
0.5199364082321195
sub-02: all
0.37789098283185923
sub-02: annotated
0.3879710302933686
sub-02: social
0.5924107057206871
sub-02: socialprimitive
0.5025188260261044
sub-02: visual
0.4337947383904805
sub-03: all
0.3607558570563405
sub-03: annotated
0.37027075265724785
sub-03: social
0.45973866149800097
sub-03: socialprimitive
0.40422288811008483
sub-03: visual
0.40035745372401305
sub-04: all
0.40334746236281094
sub-04: annotated
0.42449088842849325
sub-04: social
0.5452816896173873
sub-04: socialprimitive
0.5168589101877343
sub-04: visual
0.43964581